# Árboles de decisión

## Introducción

En este Notebook vamos a analizar el funcionamiento de los [árboles de decisión](http://scikit-learn.org/stable/modules/tree.html).


## Árboles de decisión

Es un algoritmo basado en reglas de decisión sencillas, que dividen el espacio de características en rectángulos. Es algo así como jugar al [*quién es quién*](https://es.wikipedia.org/wiki/Guess_Who%3F) con los datos.

<img src="./figs/CART_tree_titanic_survivors.png"> Fuente: [Wikipedia](https://en.wikipedia.org/wiki/Decision_tree_learning)

Existen distintos algoritmos para implementar árboles de decisión, algunos son: [ID3 (Iterative Dichotomiser)](https://en.wikipedia.org/wiki/ID3_algorithm), [C4.5](https://en.wikipedia.org/wiki/C4.5_algorithm), o CART (Classification And Regression Tree). Este último es el que utiliza scikit-learn.

CART es un algoritmo iterativo, en el que incrementalmente se divide el espacio de acuerdo con la característica o atributo que menor error de clasificación produzca al realizar dicha división. Esta división se conoce como *splitting*, y el resultado es la creación de una o varias ramas. El algoritmo de *splitting* vuelve a aplicarse sobre cada rama, hasta que en la rama el error de clasificación no mejora, y entonces se crea una hoja. 

<img src="./figs/Recursive_Splitting.png"> 
Fuente: [Wikipedia](https://en.wikipedia.org/wiki/Decision_tree_learning)

In [1]:
# cargamos librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import naive_bayes
from sklearn import svm
from sklearn import tree
import sklearn.metrics as met
from sklearn.metrics import confusion_matrix
%matplotlib inline

In [43]:
db = pd.read_csv("asset.csv")
print (db.shape)
print (db.size)

(8702, 5)
43510


In [44]:
db.head(2)

Date      Open      High       Low     Close
0  2017-03-24  29350.60  29539.85  29350.17  29421.40
1  2017-03-23  29201.01  29373.79  29198.08  29332.16

In [45]:
#REPASO
##### Method 1: Replace NaN values with 0.
db = db.fillna(0)
##### Method 2: Replace NaN values in each column with the mean of that column.
db["Open"] = db["Open"].fillna(db["Open"].mean())
db["Close"] = db["Close"].fillna(db["Close"].mean())
db["High"] = db["High"].fillna(db["High"].mean())
db["Low"] = db["Low"].fillna(db["Low"].mean())
##### Method 3: Replace NaN values in each column with the median of that column.
db["Open"] = db["Open"].fillna(db["Open"].median())
db["Close"] = db["Close"].fillna(db["Close"].median())
db["High"] = db["High"].fillna(db["High"].median())
db["Low"] = db["Low"].fillna(db["Low"].median())
##### Method 4: Replace NaN values in each column with the mode of that column.
db["Open"] = db["Open"].fillna(db["Open"].mode().mean())
db["Close"] = db["Close"].fillna(db["Close"].mode().mean())
db["High"] = db["High"].fillna(db["High"].mode().mean())
db["Low"] = db["Low"].fillna(db["Low"].mode().mean())

In [52]:
#construyo target variable
db["Change"] = db["Close"] - db["Open"]
db["label"] = db["Change"].apply(lambda x : 1 if x > 0 else -1)

In [56]:
#Añado algunas variables
db["Open1"] = db["Open"].shift(-1)
db["Close1"] = db["Close"].shift(-1)
db["High1"] = db["High"].shift(-1)
db["Low1"] = db["Low"].shift(-1)
db = db.dropna()

In [91]:
features = ["Open1", "Close1", "High1", "Low1"]

In [92]:
db[features].head(10)

Open1    Close1     High1      Low1
0  29201.01  29332.16  29373.79  29198.08
1  29341.41  29167.68  29341.41  29137.48
2  29525.88  29485.45  29585.05  29380.14
3  29653.54  29518.74  29699.48  29482.40
4  29755.74  29648.99  29824.62  29601.86
5  29482.83  29585.85  29614.79  29482.83
6  29452.86  29398.11  29500.08  29358.91
7  29437.23  29442.63  29561.93  29356.05
8  29006.00  28946.23  29076.63  28851.04
9  28909.70  28929.13  28986.72  28815.02

In [93]:
label = "label"
X = db[features][:-1]
y = db[label][:-1]

In [94]:
#creo datasets de entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [95]:
# model_1 = linear_model.LogisticRegression()
model_1 = tree.DecisionTreeClassifier()
model_1.fit(X_train, y_train)
predictions = model_1.predict(X_test)

In [96]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix

print (classification_report(y_test, predictions))
print (accuracy_score(y_test, predictions))

             precision    recall  f1-score   support

         -1       0.51      0.47      0.49       668
          1       0.68      0.71      0.70      1072

avg / total       0.62      0.62      0.62      1740

0.620689655172


In [97]:
pd.crosstab(y_test, predictions, rownames=["Actual"], colnames=["Predicted"])

Predicted   -1    1
Actual             
-1         317  351
 1         309  763